# Malaysia COVID-19 Cases and Vaccination

Data taken from [**Malaysia's Ministry of Health**](https://github.com/MoH-Malaysia/covid19-public) and [**COVID-19 Immunisation Task Force (CITF)**](https://github.com/CITF-Malaysia/citf-public) as of **`6-10-2021`** (cut-off date).

1. [covid19-public](https://github.com/MoH-Malaysia/covid19-public/tree/9a8482527c486effa37900756631f843af4d6373)
2. [citf-public](https://github.com/CITF-Malaysia/citf-public/tree/46f91d6e18e38a24034bf8f791cd279b31f94f24)